In [ ]:
#inicializando GEE
try:
    import ee
except ModuleNotFoundError: 
    !pip install earth-engine-api
    import ee
    
try:
    ee.Initialize()
except ee.EEException:
    !earthengine authenticate
    
import geemap
import folium
from ipygee import *
import matplotlib.pyplot as plt
from matplotlib import gridspec
import pandas as pd

%matplotlib inline
# install some packages
#!pip install ipygee

In [ ]:
#polygons loading hardcode
#ID411 = r"C:\Users\guilherme.fronza\OneDrive\Amazon_Select\Notebooks\ID_411.shp"
#ID373 = r"C:\Users\guilherme.fronza\OneDrive\Amazon_Select\Notebooks\ID_373.shp"
ID483 = r"C:\Users\guilherme.fronza\OneDrive\cacauFlorestaAcelerador\amazon_acelerador\04a_dados_espaciais\02_dados_scripts\source\ID_483.shp"

#create ee.Feature with GEEMAP
pol = geemap.shp_to_ee(ID483)

#NDVI to Landsat-8
def ndvi_func_LC8(image):
    ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

#NDVI to Sentinel-2
def ndvi_func_S2(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

#Modis scale factor function
def scale_factor_MODIS(image):
    # scale factor for the MODIS MOD13Q1 product
    return image.multiply(0.0001).copyProperties(image, ['system:time_start'])

##Mascará de nuvens para a banda pixel_qa SR landsat 4,5 e 7
def cloudMaskL457(image):
    qa = image.select('pixel_qa') ##substitiu a band FMASK
    cloud1 = qa.bitwiseAnd(1<<5).eq(0)
    cloud2 = qa.bitwiseAnd(1<<7).eq(0)
    cloud3 = qa.bitwiseAnd(1<<3).eq(0)
    mask2 = image.mask().reduce(ee.Reducer.min());
    return image.updateMask(cloud1).updateMask(cloud2).updateMask(cloud3).updateMask(mask2).divide(10000).copyProperties(image, ["system:time_start"])

# Define a cloud masking function.
def maskL8sr(image):
    cloudShadowBitMask = 1<<3
    cloudBitMask = 1<<5
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudBitMask).eq(0))
    return image.updateMask(mask)


#Visualization with Folium Map GEE integrated
def add_ee_layer(self, ee_object, vis_params, name):
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            fill = False,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    except:
        print("Could not display {}".format(name))

#Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

polygon_params = {
  'palette': ['red']}
location = pol.geometry().centroid(10).coordinates().getInfo()[::-1]
Map = folium.Map(location=location, zoom_start=18)
basemaps['Google Satellite Hybrid'].add_to(Map)
Map.add_ee_layer(pol, polygon_params, 'polygon')
loc = '-'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc)  
Map.get_root().html.add_child(folium.Element(title_html))
Map

In [ ]:
def TS_NDVI_plot(imagecoll, polygon, filter_param, scale):
    #time series
    polygon_ndvi = chart.Image.series(**{'imageCollection': imagecoll.select('NDVI'),
                                       'region': polygon,
                                       'reducer': ee.Reducer.mean(),
                                       'scale': scale,
                                       'xProperty': 'system:time_start'})
    df = polygon_ndvi.dataframe
    df = df.rolling(filter_param, min_periods=1).mean()
    return df


# Image collection build, spatial and temporal filter
l8sr = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
  .filterBounds(pol) \
  .filterDate('2015-01-01', '2022-12-31').map(maskL8sr)

#NDVI 
images_ndvi_LC8 = l8sr.map(ndvi_func_LC8)

#TS Constructors
LC8_NDVI_ts = TS_NDVI_plot(images_ndvi_LC8, pol, filter_param=1, scale=30)

In [ ]:
LC8_NDVI_ts

In [ ]:
LC8_NDVI_ts.plot(figsize=(15, 6))
plt.title('Série temporal NDVI Landsat-8')
plt.legend(loc='best')
plt.show()

In [ ]:
LC8_NDVI_ts.plot(figsize=(15, 6),kind='box',vert=False)
plt.title('Box-plot da Série temporal NDVI MODIS')
plt.show()

In [ ]:
#Percentual de outliers
str(round((len(LC8_NDVI_ts[LC8_NDVI_ts['NDVI']<0.6])/len(LC8_NDVI_ts))*100,2)) + " % dos dados são outliers"

In [ ]:
#enriquecendo o dataset
dataset = LC8_NDVI_ts
dataset['dia_do_ano'] = [i.dayofyear for i in dataset.index]
dataset['semana_do_ano'] = [i.weekofyear for i in dataset.index]

In [ ]:
dataset

In [ ]:
data = dataset.sample(frac=0.95, random_state=786)
data_unseen = dataset.drop(data.index)

print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

In [ ]:
data.head(5)

In [ ]:
from pycaret.anomaly import setup , create_model , assign_model , plot_model , predict_model , save_model , load_model 
exp_ano101 = setup(data, normalize = True, session_id = 123)

In [ ]:
iforest = create_model('iforest', fraction = 0.01)
iforest_results = assign_model(iforest)
iforest_results[iforest_results['Anomaly'] == 1].head()

In [ ]:
plot_model(iforest,feature="NDVI")

In [ ]:
data_unseen['NDVI'][0] = 0.5
unseen_predictions = predict_model(iforest, data=data_unseen)
unseen_predictions[unseen_predictions['Anomaly'] == 1].head()

In [ ]:
data

In [ ]:
save_model(iforest,'Anomaly_Detection_NDVI_IForest_Model')

In [ ]:
Anomaly_Detection_Model = load_model('Anomaly_Detection_NDVI_IForest_Model')

In [ ]:
anomaly = data[data['NDVI'] <0.6]
anomaly.shape

In [ ]:
data.shape

In [ ]:
new_prediction = predict_model(Anomaly_Detection_Model, data=dataset)
#Excluindo anomalias positivas
anomaly = new_prediction[new_prediction['Anomaly'] == 1][new_prediction['NDVI']<0.4]


In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))
ax.scatter(anomaly.index,anomaly['NDVI'],color='red',label='Anomalies')
ax.plot(LC8_NDVI_ts['NDVI'],label='NDVI')
plt.title('Testing anomalies detection into agroforestry areas,  Landsat-8 NDVI Time Series - 2015/2022', size=15)
plt.legend(loc='best')
plt.style.use('fivethirtyeight')
plt.show()